![DSB logo](img/Dolan.jpg)
# Assemble Data Together

## PD4E Chapter 4: Data Assembly
### How do you read/manipulate/store data in Python?

# What You Learned in Python/Pandas that could Apply Here

You will need following knowledge from the first half of this course:
1. loading data
2. subsetting data
3. functions

# What You will Learn in this Chapter
You will learn following techniques in this chapter:
1. The concept of __tidy data__
2. concatenating data
3. merging datasets

# What is 'Tidy Data'?

- In general, tidy data is a framework to structure data so that they can be easily analyzed
- Hadley Wickham defines tidy data as meeting the following requirements:
    - Each row is an observation
    - Each column is a feature
    - Each type of observation unit forms a table
    - more details are covered in Chap. 6 and more in BA 545
- Read section 4.2.1 on your own 

# Concatenation
- Concatenation can be considered as adding column/row to your data
    - this operation is useful when you try to combine different parts of data together
    - `Pandas` provides a `concat()` function for this purpose
    - in analytics, often you collect data in different parts
        - so combining them are necessary
    - in other scenarios, you may create versions of data where you perform calculations on
        - then you may want to combine the calculated results back to the original data

In [1]:
# adding new DataFrames as rows
# importing pandas first

import pandas as pd

# read in different parts of data
# please change your PATH to `'/srv/data/my_shared_data_folder/ba505-data/concat_1.csv'`
df1 = pd.read_csv('./data/concat_1.csv')
# please change your PATH to `'/srv/data/my_shared_data_folder/ba505-data/concat_2.csv'`
df2 = pd.read_csv('./data/concat_2.csv')
# please change your PATH to `'/srv/data/my_shared_data_folder/ba505-data/concat_3.csv'`
df3 = pd.read_csv('./data/concat_3.csv')

df1

,A,B,C,D
0,a0,b0,c0,d0
1,a1,b1,c1,d1
2,a2,b2,c2,d2
3,a3,b3,c3,d3


In [2]:
# concatenate data
# make sure you include different parts in a list
row_concat = pd.concat([df1, df2, df3])
row_concat

,A,B,C,D
0,a0,b0,c0,d0
1,a1,b1,c1,d1
2,a2,b2,c2,d2
3,a3,b3,c3,d3
0,a4,b4,c4,d4
1,a5,b5,c5,d5
2,a6,b6,c6,d6
3,a7,b7,c7,d7
0,a8,b8,c8,d8
1,a9,b9,c9,d9


# What Happened Above?
- `concat` blindly stack the DataFrames together
    - the row indices are stacked together
    - this will cause a problem (row indices have to be _unique_)

In [3]:
# let's subset the fourth row the concatenated DF
row_concat.iloc[3,]

A    a3
B    b3
C    c3
D    d3
Name: 3, dtype: object

In [4]:
# Create a new Series of data
new_series = pd.Series(['n1', 'n2', 'n3', 'n4'])

# let's try to concat this new series to the `df1` as a new row
pd.concat([df1, new_series])

,A,B,C,D,0
0,a0,b0,c0,d0,NaN
1,a1,b1,c1,d1,NaN
2,a2,b2,c2,d2,NaN
3,a3,b3,c3,d3,NaN
0,NaN,NaN,NaN,NaN,n1
1,NaN,NaN,NaN,NaN,n2
2,NaN,NaN,NaN,NaN,n3
3,NaN,NaN,NaN,NaN,n4


# What Happend Above?
- `NaN` is the `Pandas` telling us regarding _missing values_
    - we are going to deal with them in Chap. 5
- We hope to add a new row to a DF
    - but `concat` did not treat the new `Series` as a row
    - a new column was created and misaligned with `df1`
- the reason is that `concat` did not find a common column in `new_series`
    - not like `df1, df2, df3` that have common columns
    - `concat` treat `new_series` in a new column
    
Let's see how we can fix this.

In [5]:
# we need to add common column names 
new_row_data = pd.DataFrame([['n1', 'n2', 'n3', 'n4']],
                            columns=['A', 'B', 'C', 'D'])

# now it is recognized as a row
new_row_data

,A,B,C,D
0,n1,n2,n3,n4


In [6]:
# try concatenation again
# now it works
pd.concat([df1, new_row_data])

,A,B,C,D
0,a0,b0,c0,d0
1,a1,b1,c1,d1
2,a2,b2,c2,d2
3,a3,b3,c3,d3
0,n1,n2,n3,n4


# `concat` vs. `append`

- if you try to concatenate multiple things together, you should use `concat`
- if you just try to add one single object to a DataFrame, you can use `append`

In [7]:
# `append()` a DF to another DF
df1.append(df2)

,A,B,C,D
0,a0,b0,c0,d0
1,a1,b1,c1,d1
2,a2,b2,c2,d2
3,a3,b3,c3,d3
0,a4,b4,c4,d4
1,a5,b5,c5,d5
2,a6,b6,c6,d6
3,a7,b7,c7,d7


In [8]:
# `append()` a new row to a DF
df1.append(new_row_data)

,A,B,C,D
0,a0,b0,c0,d0
1,a1,b1,c1,d1
2,a2,b2,c2,d2
3,a3,b3,c3,d3
0,n1,n2,n3,n4


In [9]:
# `append()` a `dict` to a DF
# note that the row indices are unique now
data_dict = {'A': 'n1', 'B': 'n2', 'C': 'n3', 'D': 'n4'}

df1.append(data_dict, ignore_index=True)

,A,B,C,D
0,a0,b0,c0,d0
1,a1,b1,c1,d1
2,a2,b2,c2,d2
3,a3,b3,c3,d3
4,n1,n2,n3,n4


In [10]:
# you can use `ignore_index` with `concat` to fix the replicated row indices problem
pd.concat([df1, df2, df3], ignore_index=True)

,A,B,C,D
0,a0,b0,c0,d0
1,a1,b1,c1,d1
2,a2,b2,c2,d2
3,a3,b3,c3,d3
4,a4,b4,c4,d4
5,a5,b5,c5,d5
6,a6,b6,c6,d6
7,a7,b7,c7,d7
8,a8,b8,c8,d8
9,a9,b9,c9,d9


# Adding Columns
- Concatenate columns are very similar to concatenate rows
    - the main difference is the `axis` parameter in the `concat()` function
    - by default it is `axis = 0` 
        - so the data object is concatenated in a row-wise fashion
    - if you change it to `axis = 1` 
        - then we can add columns
    - we add _more columns_ than rows

In [11]:
# Row concatenation
col_concat = pd.concat([df1, df2, df3], axis=1)
col_concat

,A,B,C,D,A,B,C,D,A,B,C,D
0,a0,b0,c0,d0,a4,b4,c4,d4,a8,b8,c8,d8
1,a1,b1,c1,d1,a5,b5,c5,d5,a9,b9,c9,d9
2,a2,b2,c2,d2,a6,b6,c6,d6,a10,b10,c10,d10
3,a3,b3,c3,d3,a7,b7,c7,d7,a11,b11,c11,d11


In [12]:
# if we try to subset columns based on their names
# we get similar results as adding rows
col_concat['A']

,A,A,A
0,a0,a4,a8
1,a1,a5,a9
2,a2,a6,a10
3,a3,a7,a11


In [13]:
# Adding a single column to a DF is similar
# we do not have to use any function
# just directly set the values to a list, dict, or a series
col_concat['new_col'] = ['n1', 'n2', 'n3', 'n4']
col_concat

,A,B,C,D,A,B,C,D,A,B,C,D,new_col
0,a0,b0,c0,d0,a4,b4,c4,d4,a8,b8,c8,d8,n1
1,a1,b1,c1,d1,a5,b5,c5,d5,a9,b9,c9,d9,n2
2,a2,b2,c2,d2,a6,b6,c6,d6,a10,b10,c10,d10,n3
3,a3,b3,c3,d3,a7,b7,c7,d7,a11,b11,c11,d11,n4


In [14]:
# If we try to fix the dupicate column names problem
# we can use `ignore_index` again
pd.concat([df1, df2, df3], axis=1, ignore_index=True)

,0,1,2,3,4,5,6,7,8,9,10,11
0,a0,b0,c0,d0,a4,b4,c4,d4,a8,b8,c8,d8
1,a1,b1,c1,d1,a5,b5,c5,d5,a9,b9,c9,d9
2,a2,b2,c2,d2,a6,b6,c6,d6,a10,b10,c10,d10
3,a3,b3,c3,d3,a7,b7,c7,d7,a11,b11,c11,d11


# Concat with Different Indices
- So far we have assumed that the objects we want to concat has the same column/row names
- That is not always the case in real world
    - we need to address if the column/row names are not aligned

In [15]:
# concat rows with different column names
# first we pretend the three parts of the data have different column names
df1.columns=['A','B','C','D']
df2.columns=['E','F','G','H']
df3.columns=['A','H','F','C']

# let's take a look at `df2`
# note that the column names have changed
df2

,E,F,G,H
0,a4,b4,c4,d4
1,a5,b5,c5,d5
2,a6,b6,c6,d6
3,a7,b7,c7,d7


In [16]:
# now let's try to concat again
row_concat_new = pd.concat([df1, df2, df3])
row_concat_new.head()

/home/tao/anaconda3/envs/tf_gpu/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,A,B,C,D,E,F,G,H
0,a0,b0,c0,d0,NaN,NaN,NaN,NaN
1,a1,b1,c1,d1,NaN,NaN,NaN,NaN
2,a2,b2,c2,d2,NaN,NaN,NaN,NaN
3,a3,b3,c3,d3,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,a4,b4,c4,d4


# What Happened Above?

- since the columns are not natively aligned, `concat()` attempts to align them, and fill any _missing values_ with `NaN`s. 
- one way to avoid the `NaN`s is to keep only the common columns that the concatenating list shares
    - we can use a parameter `join` in `concat()` to accomplish that
    - for only keeping the common columns/rows, we will set `join=inner`
    - if we do this for `[df1. df2, df3]`, we will get an empty DF since no columns are in common
        - "in common" means that all three DFs contain the same column
    - similarly we can handle this with adding columns of different rows 
        - just add parameter `axis = 1`

In [17]:
# result in an empty DF
pd.concat([df1, df2, df3], join='inner')

""
0
1
2
3
0
1
2
3
0
1


In [18]:
# `df1` and `df3` have columns in common, 
# we can try to inner-join them
pd.concat([df1, df3], join='inner')

,A,C
0,a0,c0
1,a1,c1
2,a2,c2
3,a3,c3
0,a8,d8
1,a9,d9
2,a10,d10
3,a11,d11


# Merging Multiple Datasets

- The `inner-join` and `outer-join` concepts are borrowed from the database domain
    - we use them to merge database tables
- Combining datasets based on common row/column names are only one type 
    - sometimes we combine different data parts together based on common values
    - e.g., merging _product sales_ and _inventory_ data together using the _product\_ID_
- `Pandas` provides a function `merge()` for that

We will use sets of survey data as an example.

In [19]:
# read-in different sets
# please change your PATH to `'/srv/data/my_shared_data_folder/ba505-data/DATA'`
# change `DATA` to different file names below
person=pd.read_csv('./data/survey_person.csv')
site=pd.read_csv('./data/survey_site.csv')
survey=pd.read_csv('./data/survey_survey.csv')
visited=pd.read_csv('./data/survey_visited.csv')

In [20]:
# test whether the data is correctly read in
person

,ident,personal,family
0,dyer,William,Dyer
1,pb,Frank,Pabodie
2,lake,Anderson,Lake
3,roe,Valentina,Roerich
4,danforth,Frank,Danforth


In [21]:
# test whether the data is correctly read in
site

,name,lat,long
0,DR-1,-49.85,-128.57
1,DR-3,-47.15,-126.72
2,MSK-4,-48.87,-123.40


In [22]:
# test whether the data is correctly read in
survey.head()

,taken,person,quant,reading
0,619,dyer,rad,9.82
1,619,dyer,sal,0.13
2,622,dyer,rad,7.80
3,622,dyer,sal,0.09
4,734,pb,rad,8.41


In [23]:
# test whether the data is correctly read in
visited.head()

,ident,site,dated
0,619,DR-1,1927-02-08
1,622,DR-1,1927-02-10
2,734,DR-3,1939-01-07
3,735,DR-3,1930-01-12
4,751,DR-3,1930-02-26


# What Does `Merge` Do?

- `.merge()` is a `pandas` method
- If data is given in different parts, if we want to look at the `dates` of each `site` along with longitute(`long`) and altitude (`alt`), we need to merge different parts togehter.
- In general there are three types of merge in `pandas`
    - one-to-one merge
    - Many-to-one merge
    - Many-to-many merge

In [24]:
# One-to-one merge
# let's create a subset of `visited` first and merge it with `site`

visited_sub = visited.loc[[0, 2, 6]]
visited_sub

,ident,site,dated
0,619,DR-1,1927-02-08
2,734,DR-3,1939-01-07
6,837,MSK-4,1932-01-14


In [25]:
# we merge `site` and `visited` together
# the common columns are `name` in `site` and `site` in `visited_sub`
# note that we borrow a concept from database 
# merge counts for left and right - what left/right on the merge method
o2o = site.merge(visited_sub, left_on=['name'], right_on=['site'])
o2o

,name,lat,long,ident,site,dated
0,DR-1,-49.85,-128.57,619,DR-1,1927-02-08
1,DR-3,-47.15,-126.72,734,DR-3,1939-01-07
2,MSK-4,-48.87,-123.40,837,MSK-4,1932-01-14


## Your Turn Here

Can you tell in above `.merge()` example, which one is _left_ and which one is _right_?

In [26]:
# Many-to-one merge
# the original 'visited' DF contains duplicate values in the `site` column
# if we do not subset the DF that becomes a many-to-one merge
m2o = site.merge(visited, left_on='name', right_on='site')
m2o

,name,lat,long,ident,site,dated
0,DR-1,-49.85,-128.57,619,DR-1,1927-02-08
1,DR-1,-49.85,-128.57,622,DR-1,1927-02-10
2,DR-1,-49.85,-128.57,844,DR-1,1932-03-22
3,DR-3,-47.15,-126.72,734,DR-3,1939-01-07
4,DR-3,-47.15,-126.72,735,DR-3,1930-01-12
5,DR-3,-47.15,-126.72,751,DR-3,1930-02-26
6,DR-3,-47.15,-126.72,752,DR-3,NaN
7,MSK-4,-48.87,-123.40,837,MSK-4,1932-01-14


## Your Turn Here
Can you explain why we have a `NaN` value above?

# Many-to-many merge
 - Please refer to the example on pp.105 in PD4E
 - Explain how it works

# Your Turn Here
Finish exercises below by following instructions of each of them.

## Q1. Coding Problem

### Part 1: Concat by Rows

- Use `pd.concat` by adding the two partial DFs by rows.
- The output should look like below:
    - Pay attention to the __indices__ (non-repeat)


| <br>| employee | department  |
| --- | -------- | ----------- |
|  0  | Bob      | Accounting  |
|  1  | Jake     | Engineering |
|  2  | Lisa     | Engineering |
|  3  | Susan    | HR          |

In [27]:
# define two parts of df1
df1_1 = pd.DataFrame({'employee': ['Bob', 'Jake'],
                    'group': ['Accounting', 'Engineering']})

df1_2 = pd.DataFrame({'employee': ['Lisa', 'Sue'],
                    'group': ['Engineering', 'HR']})
print('df1_1')
print(df1_1)
print('df1_2')
print(df1_2)

df1_1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
df1_2
  employee        group
0     Lisa  Engineering
1      Sue           HR


In [28]:
# combine `df1_1` and `df1_2` into `df1`


### Part 2a: Concat by Columns
- Use `pd.concat` to merge `df2` with `df1` above as `df3`
- `df3` should look like below:

| <br>| employee | department  | hire_date |
| --- | -------- | ----------- | --------- |
|  0  | Bob      | Accounting  | 2008      |
|  1  | Jake     | Engineering | 2012      |
|  2  | Lisa     | Engineering | 2004      |
|  3  | Susan    | HR          | 2014      |

In [29]:
# define `df2`

df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
df2

,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


In [30]:
# combine `df1` and `df2` as `df3`


### Part 2b: Merge Columns with `on` keyword
- Use `pd.merge`  with the `on` keyword to merge `df2` with `df1` above as `df3a`
- `df3a` should look like below:

| <br>| employee | department  | hire_date |
| --- | -------- | ----------- | --------- |
|  0  | Bob      | Accounting  | 2008      |
|  1  | Jake     | Engineering | 2012      |
|  2  | Lisa     | Engineering | 2004      |
|  3  | Susan    | HR          | 2014      |

In [31]:
# combine `df1` and `df2` as `df3a`


### Part 3: Many-to-One Merge
- Use `pd.merge` to merge `df4` with `df3` above as `df5`
- `df5` should look like below:

| <br>| employee | department  | hire_date | supervisor |
| --- | -------- | ----------- | --------- | ---------- |
|  0  | Bob      | Accounting  | 2008      | Carly      |
|  1  | Jake     | Engineering | 2012      | Guido      |
|  2  | Lisa     | Engineering | 2004      | Guido      |
|  3  | Susan    | HR          | 2014      | Steve      |

In [32]:
# define `df4`
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})
df4

,group,supervisor
0,Accounting,Carly
1,Engineering,Guido
2,HR,Steve


In [33]:
# combine `df3` and `df4` as `df5`


### Part 4: Merge with `left_on` and `right_on` keywords

- Use `pd.merge` with the `left_on` and `right_on` keywords to merge `df6` with `df5` above as `df7`
- `df7` should look like below:

| <br>| employee | department  | hire_date | supervisor | salary |
| --- | -------- | ----------- | --------- | ---------- | ------ |
|  0  | Bob      | Accounting  | 2008      | Carly      | 70000  |
|  1  | Jake     | Engineering | 2012      | Guido      | 80000  |
|  2  | Lisa     | Engineering | 2004      | Guido      | 120000 |
|  3  | Susan    | HR          | 2014      | Steve      | 90000  |

In [34]:
# define `df6`
df6 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'salary': [70000, 80000, 120000, 90000]})
df6

,name,salary
0,Bob,70000
1,Jake,80000
2,Lisa,120000
3,Sue,90000


In [35]:
# combine `df5` and `df6` as `df7`


### Part 5: Many-to-many Merge

- Use `pd.merge` with the `left_on` and `right_on` keywords to merge `df8` with `df7` above as `df9`
- `df9` should look like below:

| <br>| employee | department  | hire_date | supervisor | salary | skills   |
| --- | -------- | ----------- | --------- | ---------- | ------ | -------- |
|  0  | Bob      | Accounting  | 2008      | Carly      | 70000  | math     |
|  1  | Bob      | Accounting  | 2008      | Carly      | 70000  | spreadsheets |
|  2  | Jake     | Engineering | 2012      | Guido      | 80000  | coding |
|  3  | Jake     | Engineering | 2012      | Guido      | 80000  | linux |
|  4  | Lisa     | Engineering | 2004      | Guido      | 120000 | coding |
| 5  | Lisa     | Engineering | 2004      | Guido      | 120000 | linux |
|  6  | Susan    | HR          | 2014      | Steve      | 90000  | spreadsheets |
|  7  | Susan    | HR          | 2014      | Steve      | 90000  | management |

In [36]:
# define `df8`
df8 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'management']})

df8

,group,skills
0,Accounting,math
1,Accounting,spreadsheets
2,Engineering,coding
3,Engineering,linux
4,HR,spreadsheets
5,HR,management


In [37]:
# combine `df7` and `df8` as `df9`


### Part 6: Overview of Combined DF

- Check the dimensionalities of `df9` (`.shape()`)
- Check the unique values and their counts of `df9.skills` (`.value_counts()`)
- check the average `salary` in `df9`

__Use the code block below for this part. __

In [38]:
# Write your code here


![DSB logo](img/Dolan.jpg)
# Assemble Data Together

## PD4E Chapter 4: Data Assembly
### How do you read/manipulate/store data in Python?

![DSB logo](img/Dolan.jpg)
# Handle Missing Values

## PD4E Chapter 5: Missing Data
### How do you read/manipulate/store data in Python?

# What You Learned in Python/Pandas that could Apply Here

You will need following knowledge from the first half of this course:
1. importing packages
2. slicing/indexing data
3. using functions/methods
4. using paramters in functions/methods

# What You will Learn in this Chapter
You will learn following techniques in this chapter:
1. what is a __missing vale__
2. How __missing values__ are generated
3. How to recode and calculate __missing data__

# What is Missing Value?
- The `NaN` values we have seen multiple times in this course is how `pandas` display __missing values__
    - There are other ways of displaying missing values: `NaN, NAN, nan`
- Missing values __do not equal to__ anything
    - `NaN` does not equal to `0`, `''` (an empty string) or `[]` (an empty list)

In [39]:
from numpy import NaN, NAN, nan
# later on if you want to use `NumPy`, 
# please use the statement below
# import numpy as np

In [40]:
# `NaN` is not `True` 
NaN == True

False

In [41]:
# `NaN` is not `False` 
NaN == False

False

In [42]:
# `NaN` is not `0` 
NaN == 0

False

In [43]:
# `NaN` is not `''` 
NaN == ''

False

In [44]:
# `NaN` is not `[]` 
NaN == []

False

In [45]:
# NaN is not even NaN !!!
NaN == NaN

False

In [46]:
# `pandas` has its own way to test missing values
pd.isna(NaN)

True

# Where do Missing Values Come From?

- Missing data are generated 
    - when we load a dataset containing missing values
    - or when we munging data like we did in Chap. 4
- Some machine learning techniques do not like missing data
    - handling missing data is an important topic covered in BA 545

# Loading Data with Missing Values

- `Pandas` will automatically detect any missing values (__NOT Recommended__)
    - For instance, in the `read_csv()` method, there are different parameters to handle missing data (pp. 111)
    - In analytical practice, since we need handle missing data caused by different reasons with different methods
    - That's why we will read in the missing values directly, and handle them later
- Other reason you would generate missing data in `pandas` include _merge data_, _user input values_, and _re-indexing_
    - we saw examples of these reasons already, if you want more examples please refer to PD4E (pp. 113 - 116)

# Working with Missing Data

- In general, there are two parts when we "work with" missing data
    - detecting missing data
    - handling missing data

# Detecting Missing Data
- There are in general two types of detecting
    - Is there any missing data in the DF?
    - How much missing data are we dealing with in the DF?
    - `pandas` provides two methods `.isna()` and `.isnul()` - which are basically the same
    - I suggest you use the former __ALWAYS__

In [47]:
# Let's load another DF with missing values
# from direct observation of the first 5 rows 
# we can see `NaN`s - but that's not always the case
# please change your PATH to `'/srv/data/my_shared_data_folder/ba505-data/country_timeseries.csv'`
ebola_df = pd.read_csv('./data/country_timeseries.csv')
ebola_df.head()

,Date,Day,Cases_Guinea,Cases_Liberia,Cases_SierraLeone,Cases_Nigeria,Cases_Senegal,Cases_UnitedStates,Cases_Spain,Cases_Mali,Deaths_Guinea,Deaths_Liberia,Deaths_SierraLeone,Deaths_Nigeria,Deaths_Senegal,Deaths_UnitedStates,Deaths_Spain,Deaths_Mali
0,1/5/2015,289,2776.0,NaN,10030.0,NaN,NaN,NaN,NaN,NaN,1786.0,NaN,2977.0,NaN,NaN,NaN,NaN,NaN
1,1/4/2015,288,2775.0,NaN,9780.0,NaN,NaN,NaN,NaN,NaN,1781.0,NaN,2943.0,NaN,NaN,NaN,NaN,NaN
2,1/3/2015,287,2769.0,8166.0,9722.0,NaN,NaN,NaN,NaN,NaN,1767.0,3496.0,2915.0,NaN,NaN,NaN,NaN,NaN
3,1/2/2015,286,NaN,8157.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3496.0,NaN,NaN,NaN,NaN,NaN,NaN
4,12/31/2014,284,2730.0,8115.0,9633.0,NaN,NaN,NaN,NaN,NaN,1739.0,3471.0,2827.0,NaN,NaN,NaN,NaN,NaN


In [48]:
# Recommended way of detecting missing values
# `True` means there are missing values
ebola_df.isna().any()

Date                   False
Day                    False
Cases_Guinea            True
Cases_Liberia           True
Cases_SierraLeone       True
Cases_Nigeria           True
Cases_Senegal           True
Cases_UnitedStates      True
Cases_Spain             True
Cases_Mali              True
Deaths_Guinea           True
Deaths_Liberia          True
Deaths_SierraLeone      True
Deaths_Nigeria          True
Deaths_Senegal          True
Deaths_UnitedStates     True
Deaths_Spain            True
Deaths_Mali             True
dtype: bool

In [49]:
# the `.info()` method is one way of detecting missing values
# not very direct
ebola_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122 entries, 0 to 121
Data columns (total 18 columns):
Date                   122 non-null object
Day                    122 non-null int64
Cases_Guinea           93 non-null float64
Cases_Liberia          83 non-null float64
Cases_SierraLeone      87 non-null float64
Cases_Nigeria          38 non-null float64
Cases_Senegal          25 non-null float64
Cases_UnitedStates     18 non-null float64
Cases_Spain            16 non-null float64
Cases_Mali             12 non-null float64
Deaths_Guinea          92 non-null float64
Deaths_Liberia         81 non-null float64
Deaths_SierraLeone     87 non-null float64
Deaths_Nigeria         38 non-null float64
Deaths_Senegal         22 non-null float64
Deaths_UnitedStates    18 non-null float64
Deaths_Spain           16 non-null float64
Deaths_Mali            12 non-null float64
dtypes: float64(16), int64(1), object(1)
memory usage: 17.2+ KB


In [50]:
# following is the recommended way
# each value refers to how many missing values in the column
ebola_df.isna().sum()

Date                     0
Day                      0
Cases_Guinea            29
Cases_Liberia           39
Cases_SierraLeone       35
Cases_Nigeria           84
Cases_Senegal           97
Cases_UnitedStates     104
Cases_Spain            106
Cases_Mali             110
Deaths_Guinea           30
Deaths_Liberia          41
Deaths_SierraLeone      35
Deaths_Nigeria          84
Deaths_Senegal         100
Deaths_UnitedStates    104
Deaths_Spain           106
Deaths_Mali            110
dtype: int64

In [51]:
# We can even look at the ratio of missingness 
# Why following code works?
(ebola_df.isna().sum()/ebola_df.shape[0]).round(4) * 100

Date                    0.00
Day                     0.00
Cases_Guinea           23.77
Cases_Liberia          31.97
Cases_SierraLeone      28.69
Cases_Nigeria          68.85
Cases_Senegal          79.51
Cases_UnitedStates     85.25
Cases_Spain            86.89
Cases_Mali             90.16
Deaths_Guinea          24.59
Deaths_Liberia         33.61
Deaths_SierraLeone     28.69
Deaths_Nigeria         68.85
Deaths_Senegal         81.97
Deaths_UnitedStates    85.25
Deaths_Spain           86.89
Deaths_Mali            90.16
dtype: float64

In [52]:
# or an easier solution
ebola_df.isna().mean().round(4) * 100

Date                    0.00
Day                     0.00
Cases_Guinea           23.77
Cases_Liberia          31.97
Cases_SierraLeone      28.69
Cases_Nigeria          68.85
Cases_Senegal          79.51
Cases_UnitedStates     85.25
Cases_Spain            86.89
Cases_Mali             90.16
Deaths_Guinea          24.59
Deaths_Liberia         33.61
Deaths_SierraLeone     28.69
Deaths_Nigeria         68.85
Deaths_Senegal         81.97
Deaths_UnitedStates    85.25
Deaths_Spain           86.89
Deaths_Mali            90.16
dtype: float64

# Handling Missing Data
- There are a few strategy we can handle missing data:
    - Recode/Replace: set the missing value with another value
    - Fill forward: use the last known value to replace the missing value
    - Fill backward: use the next known value to replace the missing value
    - Interpolate: fills missing values linearly
    - Drop missing values: remove the row/column containing the missing value(s)
        - only do this when none of above works
- More to come in BA 545 
    - handling missing data is a __must-have__ step

In [53]:
# Replace missing values with fixed value `0`
# the term is called 'impute'
ebola_df_imputed = ebola_df.fillna(0)
ebola_df_imputed.head().iloc[:,:5]

,Date,Day,Cases_Guinea,Cases_Liberia,Cases_SierraLeone
0,1/5/2015,289,2776.0,0.0,10030.0
1,1/4/2015,288,2775.0,0.0,9780.0
2,1/3/2015,287,2769.0,8166.0,9722.0
3,1/2/2015,286,0.0,8157.0,0.0
4,12/31/2014,284,2730.0,8115.0,9633.0


In [54]:
# normally when we deal with continuous values here
# we use mean/average to replace missing values
ebola_guinea = ebola_df.Cases_Guinea.fillna(ebola_df.Cases_Guinea.mean())
ebola_guinea.head(10)

0    2776.000000
1    2775.000000
2    2769.000000
3     911.064516
4    2730.000000
5    2706.000000
6    2695.000000
7    2630.000000
8    2597.000000
9    2571.000000
Name: Cases_Guinea, dtype: float64

In [55]:
# fill forward
# Why we still have missing values here?
ebola_df_ffill = ebola_df.fillna(method='ffill')
ebola_df_ffill.head().iloc[:,:5]

,Date,Day,Cases_Guinea,Cases_Liberia,Cases_SierraLeone
0,1/5/2015,289,2776.0,NaN,10030.0
1,1/4/2015,288,2775.0,NaN,9780.0
2,1/3/2015,287,2769.0,8166.0,9722.0
3,1/2/2015,286,2769.0,8157.0,9722.0
4,12/31/2014,284,2730.0,8115.0,9633.0


In [56]:
# fill backward
# Why we still have missing values here?
ebola_df_bfill = ebola_df.fillna(method='bfill')
ebola_df_bfill.tail().iloc[:,:5]

,Date,Day,Cases_Guinea,Cases_Liberia,Cases_SierraLeone
117,3/27/2014,5,103.0,8.0,6.0
118,3/26/2014,4,86.0,NaN,NaN
119,3/25/2014,3,86.0,NaN,NaN
120,3/24/2014,2,86.0,NaN,NaN
121,3/22/2014,0,49.0,NaN,NaN


In [57]:
# interpolate
# Why we still have missing values here?
ebola_df.interpolate().iloc[:10, :10]

,Date,Day,Cases_Guinea,Cases_Liberia,Cases_SierraLeone,Cases_Nigeria,Cases_Senegal,Cases_UnitedStates,Cases_Spain,Cases_Mali
0,1/5/2015,289,2776.0,NaN,10030.0,NaN,NaN,NaN,NaN,NaN
1,1/4/2015,288,2775.0,NaN,9780.0,NaN,NaN,NaN,NaN,NaN
2,1/3/2015,287,2769.0,8166.0,9722.0,NaN,NaN,NaN,NaN,NaN
3,1/2/2015,286,2749.5,8157.0,9677.5,NaN,NaN,NaN,NaN,NaN
4,12/31/2014,284,2730.0,8115.0,9633.0,NaN,NaN,NaN,NaN,NaN
5,12/28/2014,281,2706.0,8018.0,9446.0,NaN,NaN,NaN,NaN,NaN
6,12/27/2014,280,2695.0,7997.5,9409.0,NaN,NaN,NaN,NaN,NaN
7,12/24/2014,277,2630.0,7977.0,9203.0,NaN,NaN,NaN,NaN,NaN
8,12/21/2014,273,2597.0,7919.5,9004.0,NaN,NaN,NaN,NaN,NaN
9,12/20/2014,272,2571.0,7862.0,8939.0,NaN,NaN,NaN,NaN,NaN


In [58]:
# Drop missing values
ebola_deaths_mali = ebola_df.Deaths_Mali
ebola_deaths_mali.shape

(122,)

In [59]:
ebola_deaths_mali_dropna = ebola_deaths_mali.dropna()
ebola_deaths_mali_dropna.shape

(12,)

In [60]:
# let's test if we have any missing values retained
ebola_deaths_mali_dropna.isnull().any()

False

# Your Turn Here
Finish exercises below by following instructions of each of them.

## Q1. Coding Problem

Detecting and handling the missing data in the _weather_ dataset.

### Part 1: Detecting Missing Data
1. Find the absolute numbers of missing values in each column of the _weather_ dataset.
2. Find the percentages of missing values in each column of the _weather_ dataset.

In [61]:
# Read-in data
# please change your PATH to `'/srv/data/my_shared_data_folder/ba505-data/weather.csv'`
weather_data = pd.read_csv('./data/weather.csv')
weather_data.head()

,id,year,month,element,d1,d2,d3,d4,d5,d6,...,d22,d23,d24,d25,d26,d27,d28,d29,d30,d31
0,MX17004,2010,1,tmax,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.8,NaN
1,MX17004,2010,1,tmin,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.5,NaN
2,MX17004,2010,2,tmax,NaN,27.3,24.1,NaN,NaN,NaN,...,NaN,29.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MX17004,2010,2,tmin,NaN,14.4,14.4,NaN,NaN,NaN,...,NaN,10.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MX17004,2010,3,tmax,NaN,NaN,NaN,NaN,32.1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
# write your code here


### Part 2: Handling Missing Data
1. Replace missing values in column `d5` with the mean/average of the column.
2. Forward fill missing values in column `d14`.
3. Backward fill missing values in column `d2`.
4. Interpolate missing values in column `d3`.
5. Drop columns is the missing value ratio is at `100%`.

__NOTE__: create a copy of the dataset `weather_data_copy` for these tasks.

In [63]:
# write your code here


### Part 3: Checking Missing Data Again
1. Check the percentages of missing values of each column in `weather_data_copy` after __imputation__.

In [64]:
# write your code here


# Classwork (start here in class)
You can start working on them right now:
- Read Chapters 4 & 5 in PD4E 
- If time permits, start in on your homework. 
- Ask questions when you need help. Use this time to get help from the professor!

# Homework (do at home)
The following is due before class next week:
  - Any remaining classwork from tonight
  - DataCamp “Visualizing a Categorical and a Quantitative Variable” assignment

Note: All work on DataCamp is logged. Don't try to fake it!

Please email [me](mailto:jtao@fairfield.edu) if you have any problems or questions.

![DSB logo](img/Dolan.jpg)
# Handle Missing Values

## PD4E Chapter 5: Missing Data
### How do you read/manipulate/store data in Python?